In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options

option = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values":{"notifications":2}}
option.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome("C:\\Users\\CCE\\Downloads\\chromedriver.exe",options=option)
driver.maximize_window()

driver.get("https://www.cwb.gov.tw/V8/C/C/Statistics/monthlydata.html")

driver.refresh()

time.sleep(1)

In [ ]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import numpy as np
import pandas as pd
import os

def mkdir(path):

    if  os.path.exists(path) == False:
        os.makedirs(path)
        print('mkdir:',path)
    elif  os.path.exists(path) == True:
        print('dir already exist:',path)

year = Select(driver.find_element_by_name("year"))

years = []
months = []

for op in year.options:
    years.append(op.text)

years.remove("2022")
year.select_by_value("2021")

month = Select(driver.find_element_by_name("month"))
for op2 in month.options:
    months.append(op2.text)

length=1
# soup = BeautifulSoup(driver.page_source,"html.parser")
while length != len(years):    
    year.select_by_value(years[length-1])
    
    for i in range(len(months)):
        datalist=[]
        month.select_by_value(months[i])
        
        header = [j.text for j in driver.find_elements_by_css_selector("thead>tr[class='th_row_2']>th") if j.text]
        header[1] = header[1].replace("平均","平均溫度")
        header[-5] = header[-5].replace("平均","平均相對溼度")
        
        rows = [a for a in driver.find_elements_by_css_selector("tbody[id='MonthlyData_MOD']>tr") if a.text]
        
        sequel=0
        for row in rows:
            if row.find_elements_by_css_selector("th") != None:
                sequel+=1
            
            datalist.append([q.text for q in row.find_elements_by_css_selector("th,td") if q.text])
         
        df = pd.DataFrame(datalist,columns = header,)
        df.insert(0,column="日期",value = [years[length-1]+"/"+months[i]]*sequel)

        # excel存放的資料夾
        file_dir="C:/Users/CCE"
        # 若資料夾不存在，建立指定資料夾
        mkdir(file_dir)
        file_out=file_dir+"/weather.xlsx"
        # 如果檔案不存在，建立檔案
        if  os.path.exists(file_out) == False:
            df.to_excel(file_out,index=False)
        # 如果檔案存在，append
        elif  os.path.exists(file_out) == True:
            df_old = pd.read_excel(file_out)
            
            df_combine = pd.concat([df,df_old],axis = 0)

            df_combine.to_excel(file_out,index=False)
               
        print("已完成%s年%s月的資料" % (years[length-1],months[i]))
        print("-----------------------------------------------------------------------------------------------")
        time.sleep(10)
    
    length+=1

driver.close()




